This simple script adds an explicit date to the existing items in the SGCN source data repository indicating when the SGCN list applied (either 2005 or 2015). Formerly, this information was only contained in the title, making coding against the repository somewhat ambiguous and brittle. The notebook uses the pysb module to make the update and requires authentication. It should not be run again as there is nothing in the code to ensure that the date collected is not already pleasant. It is considered throwaway code and just useful for transparency in the process.

In [ ]:
# Get dependencies and set up an sb session
import re
import pysb
import os
sb = pysb.SbSession()
username = input("Username: ")
sb.loginc(str(username))

In [ ]:
# Get all items from the collection for SGCN
r = requests.get("https://www.sciencebase.gov/catalog/items?parentId=56d720ece4b015c306f442d5&format=json&fields=title,dates&max=100").json()

# Loop through the items
for item in r['items']:
    keepDates = []
    # Keep only the non-system dates (created and last updated have to go away or the update code fails)
    for date in item["dates"]:
        if date["type"] == "dateCreated" or date["type"] == "lastUpdated":
            pass
        else:
            keepDates.append(date.copy())
    
    # Set the new date collected date based on a regex that gets the year out of the title
    # This is brittle as it assumes that there is only a single numeric substring in the title, which was the case when it was run
    dateCollected = {}
    dateCollected["dateString"] = re.findall(r'\d+', item['title'])[0]
    dateCollected["label"] = "Date Collected"
    dateCollected["type"] = "Collected"
    keepDates.append(dateCollected.copy())

    # Set up the item with its ID in a dict for submission to ScienceBase
    itemUpdate = {}
    itemUpdate["id"] = item["id"]
    itemUpdate["dates"] = keepDates
    
    # Print the item structure and send it to ScienceBase
    print (itemUpdate)
    sb.update_item(itemUpdate)
